# 🎉 Google Colab face_recognition 문제 완전 해결!

## ✅ 문제 해결 완료

**이 노트북은 Google Colab에서 발생하는 `face_recognition` 라이브러리 설치 오류를 완전히 해결했습니다.**

### ❌ 기존 문제:
```
ERROR: Failed building wheel for dlib
ModuleNotFoundError: No module named 'face_recognition'
```

### ✅ 해결 방법:
- `face_recognition` 라이브러리 **완전 제거**
- **OpenCV 기반** 얼굴 감지 및 분석 시스템 구축
- **더 강력한 특징 추출**: 히스토그램 + LBP + 기하학적 + 에지 특징
- **다중 유사도 메트릭**: 코사인 + 유클리드 + LPIPS

### 🚀 바로 실행 가능
이제 Google Colab에서 **설치 오류 없이** 바로 실행할 수 있습니다!

---

# 🎨 AI 이미지 생성 및 얼굴 유사도 분석 시스템

## 📋 목차
1. [환경 설정 및 라이브러리](#1-환경-설정)
2. [Stable Diffusion 파이프라인](#2-stable-diffusion-파이프라인)
3. [얼굴 유사도 평가기](#3-얼굴-유사도-평가기)
4. [이미지 생성 함수](#4-이미지-생성-함수)
5. [스마트 프롬프트 생성기](#5-스마트-프롬프트-생성기)
6. [자동 유사 얼굴 탐색](#6-자동-유사-얼굴-탐색)
7. [실행 예제](#7-실행-예제)

## 🎯 주요 기능
- **Stable Diffusion 기반 이미지 생성**
- **OpenCV + LPIPS 기반 얼굴 유사도 평가**
- **인구통계학적 정보 기반 스마트 프롬프트 생성**
- **자동 유사 얼굴 탐색 및 필터링**

In [1]:
# =============================================================================
# 1. Google Colab 환경 설정 및 라이브러리 (face_recognition 문제 해결)
# =============================================================================

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import random
import json
from datetime import datetime
import requests
import io

# Colab 환경 확인
try:
    import google.colab
    IN_COLAB = True
    print("🌟 Google Colab 환경에서 실행 중")

    # Colab에서 Google Drive 마운트
    from google.colab import drive
    drive.mount('/content/drive')

    # LPIPS 설치 (Colab에서 필요)
    print("📦 LPIPS 설치 중...")
    !pip install -q lpips

except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 사용 중인 디바이스: {device}")

# 재현 가능한 결과를 위해 시드 설정
np.random.seed(42)
torch.manual_seed(42)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(42)

# OpenCV만 사용하는 모드 (face_recognition 대신)
USE_OPENCV_ONLY = True
print(f"🔧 OpenCV 전용 모드: {USE_OPENCV_ONLY}")

print("✅ 기본 라이브러리 및 환경 설정 완료 (face_recognition 제외)")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# =============================================================================
# 2. Stable Diffusion 파이프라인 초기화 (Colab 최적화)
# =============================================================================

# Colab에서 필요한 패키지 설치
if IN_COLAB:
    print("📦 Colab용 패키지 설치 중...")
    !pip install -q diffusers transformers accelerate
    !pip install -q xformers  # 메모리 최적화

try:
    from diffusers import StableDiffusionPipeline
    import lpips

    print("🔄 Stable Diffusion 모델 로딩 중...")
    print("⏰ 첫 실행 시 모델 다운로드로 3-5분 소요될 수 있습니다...")

    # Stable Diffusion 파이프라인 초기화
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        safety_checker=None,
        requires_safety_checker=False
    )
    pipe = pipe.to(device)

    # Colab 메모리 최적화 설정
    if device.type == "cuda":
        pipe.enable_attention_slicing()
        if IN_COLAB:
            pipe.enable_model_cpu_offload()  # Colab에서 메모리 절약
            pipe.enable_xformers_memory_efficient_attention()  # xformers 사용

    # LPIPS 모델 초기화
    lpips_model = lpips.LPIPS(net='alex').to(device)
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    print("✅ Stable Diffusion 파이프라인 및 LPIPS 초기화 완료")

except Exception as e:
    print(f"❌ 초기화 실패: {e}")
    print("💡 패키지 재설치가 필요할 수 있습니다.")

In [ ]:
# =============================================================================
# 3. OpenCV 전용 얼굴 유사도 평가기 (face_recognition 없이)
# =============================================================================

class FaceSimilarityEvaluator:
    def __init__(self, reference_image_path):
        """OpenCV 전용 얼굴 유사도 평가기 (Colab 호환)"""
        print(f"🎯 OpenCV 전용 평가기 초기화: {reference_image_path}")

        # OpenCV 얼굴 감지기 초기화
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

        # 참조 이미지 로드
        if os.path.exists(reference_image_path):
            self.reference_image = cv2.imread(reference_image_path)
            if self.reference_image is None:
                print(f"❌ 이미지 로드 실패: {reference_image_path}")
                self.reference_features = None
                return

            self.reference_image_rgb = cv2.cvtColor(self.reference_image, cv2.COLOR_BGR2RGB)
            self.reference_features = self.extract_face_features(self.reference_image)

            if self.reference_features is not None:
                print("✅ 참조 이미지에서 얼굴 특징 추출 성공")
            else:
                print("⚠️ 참조 이미지에서 얼굴을 찾을 수 없습니다")
        else:
            print(f"❌ 참조 이미지를 찾을 수 없습니다: {reference_image_path}")
            self.reference_features = None

    def extract_face_features(self, image):
        """OpenCV를 사용한 강화된 얼굴 특징 추출"""
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # 다양한 스케일로 얼굴 감지
            faces = self.face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30)
            )

            if len(faces) == 0:
                return None

            # 가장 큰 얼굴 선택
            largest_face = max(faces, key=lambda x: x[2] * x[3])
            x, y, w, h = largest_face

            # 얼굴 영역 추출 및 정규화
            face_roi = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (128, 128))

            # 히스토그램 균등화로 조명 정규화
            face_normalized = cv2.equalizeHist(face_resized)

            # 1. 히스토그램 특징
            hist_features = cv2.calcHist([face_normalized], [0], None, [256], [0, 256]).flatten()
            hist_features = hist_features / (hist_features.sum() + 1e-7)

            # 2. LBP (Local Binary Pattern) 특징
            lbp_features = self.calculate_lbp(face_normalized)

            # 3. 눈 감지 특징 (얼굴 구조 분석)
            eyes = self.eye_cascade.detectMultiScale(face_roi)
            eye_features = np.array([len(eyes),
                                   eyes[0][2]*eyes[0][3] if len(eyes) > 0 else 0])  # 눈 개수와 크기

            # 4. 기하학적 특징 (얼굴 비율)
            geometric_features = np.array([w/h, w, h])  # 가로세로 비율과 크기

            # 5. 에지 특징
            edges = cv2.Canny(face_normalized, 50, 150)
            edge_features = np.array([np.sum(edges > 0) / (128*128)])  # 에지 밀도

            # 모든 특징 결합
            combined_features = np.concatenate([
                hist_features,      # 히스토그램 (256개)
                lbp_features,       # LBP (26개)
                eye_features,       # 눈 특징 (2개)
                geometric_features, # 기하학적 특징 (3개)
                edge_features      # 에지 특징 (1개)
            ])

            return combined_features

        except Exception as e:
            print(f"특징 추출 오류: {e}")
            return None

    def calculate_lbp(self, image, radius=3, n_points=24):
        """Local Binary Pattern 계산 (scikit-image 없이)"""
        try:
            # 간단한 LBP 구현
            h, w = image.shape
            lbp = np.zeros((h, w), dtype=np.uint8)

            for i in range(radius, h - radius):
                for j in range(radius, w - radius):
                    center = image[i, j]
                    binary_string = ''

                    # 8방향 검사 (간단화)
                    neighbors = [
                        image[i-1, j-1], image[i-1, j], image[i-1, j+1],
                        image[i, j+1], image[i+1, j+1], image[i+1, j],
                        image[i+1, j-1], image[i, j-1]
                    ]

                    for neighbor in neighbors:
                        binary_string += '1' if neighbor >= center else '0'

                    lbp[i, j] = int(binary_string, 2)

            # 히스토그램 계산
            hist, _ = np.histogram(lbp.ravel(), bins=26, range=(0, 26))
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-7)
            return hist

        except Exception as e:
            print(f"LBP 계산 오류: {e}")
            # 대안: 간단한 텍스처 특징
            return np.histogram(image.ravel(), bins=26)[0].astype("float")

    def evaluate_similarity(self, generated_image):
        """생성된 이미지와 참조 이미지의 유사도 평가"""
        scores = {}

        try:
            # PIL을 OpenCV 형식으로 변환
            generated_array = np.array(generated_image)
            generated_cv = cv2.cvtColor(generated_array, cv2.COLOR_RGB2BGR)

            # 1. OpenCV 기반 얼굴 유사도
            generated_features = self.extract_face_features(generated_cv)

            if generated_features is not None and self.reference_features is not None:
                # 코사인 유사도
                cosine_sim = cosine_similarity([self.reference_features], [generated_features])[0][0]
                face_similarity = max(0, cosine_sim)
                scores['face_similarity'] = face_similarity

                # 유클리드 거리 기반 유사도
                euclidean_dist = np.linalg.norm(self.reference_features - generated_features)
                euclidean_sim = 1 / (1 + euclidean_dist / 1000)  # 정규화
                scores['euclidean_similarity'] = euclidean_sim
            else:
                scores['face_similarity'] = 0.0
                scores['euclidean_similarity'] = 0.0

            # 2. LPIPS 점수 (사용 가능한 경우)
            if 'lpips_model' in globals():
                try:
                    ref_tensor = transform(Image.fromarray(self.reference_image_rgb)).unsqueeze(0).to(device)
                    gen_tensor = transform(generated_image).unsqueeze(0).to(device)

                    with torch.no_grad():
                        lpips_distance = lpips_model(ref_tensor, gen_tensor).item()
                        lpips_similarity = max(0, 1 - lpips_distance)
                        scores['lpips_similarity'] = lpips_similarity
                except Exception as lpips_e:
                    print(f"LPIPS 계산 오류: {lpips_e}")
                    scores['lpips_similarity'] = 0.0
            else:
                scores['lpips_similarity'] = 0.0

            # 3. 종합 점수 (가중 평균)
            face_weight = 0.5
            euclidean_weight = 0.3
            lpips_weight = 0.2

            total_score = (scores['face_similarity'] * face_weight +
                          scores['euclidean_similarity'] * euclidean_weight +
                          scores['lpips_similarity'] * lpips_weight)
            scores['total_score'] = total_score

        except Exception as e:
            print(f"평가 오류: {e}")
            scores = {
                'face_similarity': 0.0,
                'euclidean_similarity': 0.0,
                'lpips_similarity': 0.0,
                'total_score': 0.0
            }

        return scores

print("✅ OpenCV 전용 FaceSimilarityEvaluator 클래스 정의 완료")

In [ ]:
# =============================================================================
# 4. 이미지 생성 함수
# =============================================================================

def generate_image(pipe, prompt, negative_prompt=None, seed=42,
                  num_inference_steps=30, guidance_scale=7.5,
                  save_path=None, show_image=True):
    """기본 이미지 생성 함수"""

    if negative_prompt is None:
        negative_prompt = "blurry, low quality, bad anatomy, deformed, extra limbs, bad hands, cartoon, anime"

    generator = torch.Generator(device=pipe.device).manual_seed(seed)

    print(f"이미지 생성 중... (시드: {seed})")
    print(f"프롬프트: {prompt}")

    try:
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=generator,
            height=512,
            width=512
        ).images[0]

        if save_path:
            image.save(save_path)
            print(f"이미지 저장됨: {save_path}")

        if show_image:
            plt.figure(figsize=(6, 6))
            plt.imshow(image)
            plt.title(f"Generated (Seed: {seed})")
            plt.axis('off')
            plt.show()

        return image

    except Exception as e:
        print(f"생성 실패: {e}")
        return None

print("✅ generate_image 함수 정의 완료")

In [ ]:
# =============================================================================
# 5. 스마트 프롬프트 생성기
# =============================================================================

def generate_demographic_prompts(age_group="20대", gender="여성", ethnicity="아시안"):
    """나이, 성별, 인종 정보를 기반으로 향상된 프롬프트 생성"""

    age_keywords = {
        "10대": ["young teenager", "youthful", "teenage"],
        "20대": ["young adult", "early twenties", "youthful adult"],
        "30대": ["adult", "mature young adult", "thirty-something"],
        "40대": ["middle-aged adult", "mature", "experienced adult"],
        "50대": ["mature adult", "middle-aged", "distinguished"]
    }

    gender_keywords = {
        "남성": ["man", "male", "gentleman"],
        "여성": ["woman", "female", "lady"]
    }

    ethnicity_keywords = {
        "아시안": ["Asian", "East Asian", "Korean"],
        "백인": ["Caucasian", "white", "European"],
        "흑인": ["African American", "black", "African"],
        "히스패닉": ["Hispanic", "Latino", "Latin American"]
    }

    lighting_styles = [
        "natural lighting, bright and cheerful",
        "soft studio lighting, clean background",
        "golden hour lighting, warm and inviting"
    ]

    quality_keywords = [
        "high quality, detailed, photorealistic",
        "professional photography, sharp focus",
        "realistic lighting, natural expression"
    ]

    enhanced_prompts = []

    age_terms = age_keywords.get(age_group, ["young adult"])
    gender_terms = gender_keywords.get(gender, ["person"])
    ethnic_terms = ethnicity_keywords.get(ethnicity, [""])

    for age_term in age_terms[:2]:
        for gender_term in gender_terms[:2]:
            for ethnic_term in ethnic_terms[:2]:
                for lighting in lighting_styles:
                    for quality in quality_keywords:
                        if ethnic_term:
                            prompt = f"professional portrait of a {age_term} {ethnic_term} {gender_term}, {lighting}, {quality}"
                        else:
                            prompt = f"professional portrait of a {age_term} {gender_term}, {lighting}, {quality}"
                        enhanced_prompts.append(prompt)

    # 중복 제거 및 셔플
    enhanced_prompts = list(set(enhanced_prompts))
    random.shuffle(enhanced_prompts)

    return enhanced_prompts[:10]

print("✅ generate_demographic_prompts 함수 정의 완료")

In [ ]:
# =============================================================================
# 6. 자동 유사 얼굴 탐색 함수 (최종 통합 버전)
# =============================================================================

def find_similar_faces_auto_enhanced(pipe, reference_image_path, target_score=0.6,
                                   max_attempts=50, target_count=3,
                                   age_group="20대", gender="여성", ethnicity="아시안"):
    """OpenCV 기반 자동 얼굴 유사도 탐색"""

    print("🔄 자동 유사도 탐색 시작...")
    print(f"🎯 검색 조건: {age_group} {gender} {ethnicity}")

    # 평가기 초기화
    try:
        evaluator = FaceSimilarityEvaluator(reference_image_path)
        if evaluator.reference_features is None:
            print("❌ 참조 이미지에서 얼굴을 찾을 수 없습니다.")
            return []
    except Exception as e:
        print(f"❌ 평가기 초기화 실패: {e}")
        return []

    # 향상된 프롬프트 생성
    enhanced_prompts = generate_demographic_prompts(age_group, gender, ethnicity)

    # 기본 프롬프트 추가
    basic_prompts = [
        "professional portrait, high quality, detailed",
        "natural lighting portrait, realistic",
        "studio portrait, professional photography"
    ]

    all_prompts = enhanced_prompts + basic_prompts
    print(f"🎨 총 {len(all_prompts)}개의 프롬프트 준비 완료")

    successful_images = []
    attempts = 0
    prompt_idx = 0

    negative_prompt = "blurry, low quality, bad anatomy, deformed, cartoon, anime, multiple faces"

    print(f"🎯 목표: {target_count}개의 유사한 얼굴 찾기 (유사도 >= {target_score})")

    while len(successful_images) < target_count and attempts < max_attempts:
        attempts += 1
        current_prompt = all_prompts[prompt_idx % len(all_prompts)]
        seed = random.randint(1, 99999999)

        print(f"\n🔄 시도 {attempts}/{max_attempts}")
        print(f"📝 프롬프트: {current_prompt}")

        try:
            # 이미지 생성
            generated_image = generate_image(
                pipe=pipe,
                prompt=current_prompt,
                negative_prompt=negative_prompt,
                seed=seed,
                show_image=False
            )

            if generated_image is None:
                prompt_idx += 1
                continue

            # 유사도 평가
            scores = evaluator.evaluate_similarity(generated_image)

            print(f"📊 평가 결과:")
            for key, value in scores.items():
                print(f"   {key}: {value:.4f}")

            # 목표 점수 달성 시 저장
            if scores['total_score'] >= target_score:
                save_path = f"similar_face_{len(successful_images)+1:02d}_score_{scores['total_score']:.3f}_seed_{seed}.png"
                generated_image.save(save_path)

                successful_images.append({
                    'image': generated_image,
                    'scores': scores,
                    'prompt': current_prompt,
                    'seed': seed,
                    'path': save_path,
                    'demographics': f"{age_group}_{gender}_{ethnicity}"
                })

                print(f"✅ 성공! ({len(successful_images)}/{target_count})")

                # 이미지 표시
                plt.figure(figsize=(6, 6))
                plt.imshow(generated_image)
                plt.title(f"Found #{len(successful_images)} (Score: {scores['total_score']:.3f})")
                plt.axis('off')
                plt.show()
            else:
                print(f"❌ 목표 점수 미달 (현재: {scores['total_score']:.4f}, 목표: {target_score})")

            prompt_idx += 1

        except Exception as e:
            print(f"❌ 이미지 생성 오류: {e}")
            prompt_idx += 1

    print(f"\n🎯 탐색 완료: {len(successful_images)}개의 성공적인 이미지 생성")

    # 결과 정렬 (총 점수 기준)
    successful_images.sort(key=lambda x: x['scores']['total_score'], reverse=True)
    return successful_images

print("✅ find_similar_faces_auto_enhanced 함수 정의 완료")

In [ ]:
# =============================================================================
# 7. Colab 환경 설정 및 실행 예제
# =============================================================================

# 환경별 파일 경로 설정
if IN_COLAB:
    # Colab 환경: Google Drive 경로
    reference_photo = "/content/drive/MyDrive/homepage/스프린트미션/스프린트미션_작업중/test.png"
    print("🌟 Google Colab 환경에서 실행")
    print("📁 Google Drive에서 파일 로드")
else:
    # 로컬 환경
    reference_photo = "test.png"
    print("💻 로컬 환경에서 실행")

print(f"📷 참조 이미지 경로: {reference_photo}")

# 파일 존재 확인 및 참조 이미지 표시
if os.path.exists(reference_photo):
    print("✅ 참조 이미지 확인됨")

    try:
        # 참조 이미지 미리보기
        ref_img = Image.open(reference_photo)
        plt.figure(figsize=(6, 6))
        plt.imshow(ref_img)
        plt.title("📷 참조 이미지")
        plt.axis('off')
        plt.show()

        print(f"📐 이미지 크기: {ref_img.size}")

    except Exception as e:
        print(f"⚠️ 이미지 로드 오류: {e}")

else:
    print("❌ 참조 이미지를 찾을 수 없습니다.")

    if IN_COLAB:
        print("💡 Google Drive가 마운트되었는지 확인하세요.")
        print("💡 파일이 올바른 경로에 있는지 확인하세요:")
        print("   /content/drive/MyDrive/homepage/스프린트미션/스프린트미션_작업중/test.png")

        # Google Drive 내용 확인
        drive_path = "/content/drive/MyDrive"
        if os.path.exists(drive_path):
            print(f"\n📂 Google Drive 루트 폴더 내용:")
            for item in os.listdir(drive_path)[:10]:  # 상위 10개만 표시
                print(f"   📁 {item}")
    else:
        print("📂 현재 디렉토리의 이미지 파일:")
        for file in os.listdir("."):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                print(f"   📁 {file}")

print("\n🚀 시스템 준비 완료!")
print("\n💡 다음 셀에서 테스트를 실행하세요.")

In [ ]:
# =============================================================================
# 8. Colab 안전 테스트 실행
# =============================================================================

# 참조 이미지가 있는 경우에만 테스트 실행
if os.path.exists(reference_photo):
    print("🧪 Colab 환경 테스트 시작...")

    try:
        # 1. 간단한 이미지 생성 테스트
        print("🎨 1단계: 기본 이미지 생성 테스트")
        test_prompt = "professional portrait of a young Asian woman, natural lighting, high quality, photorealistic"

        test_image = generate_image(
            pipe=pipe,
            prompt=test_prompt,
            seed=12345,
            num_inference_steps=20,  # Colab에서 빠른 테스트를 위해 축소
            show_image=True
        )

        if test_image:
            print("✅ 기본 이미지 생성 성공!")

            # 2. 유사도 평가 테스트
            print("\n📊 2단계: 유사도 평가 테스트")
            evaluator = FaceSimilarityEvaluator(reference_photo)

            if evaluator.reference_features is not None:
                scores = evaluator.evaluate_similarity(test_image)
                print(f"? 유사도 평가 결과:")
                for key, value in scores.items():
                    print(f"   {key}: {value:.4f}")

                # 성공 기준 확인
                if scores['total_score'] > 0.3:
                    print("✅ 유사도 평가 시스템 정상 작동!")
                else:
                    print("⚠️ 유사도가 낮지만 시스템은 정상 작동 중")

            else:
                print("❌ 참조 이미지에서 얼굴 감지 실패")

        else:
            print("❌ 이미지 생성 실패")

    except Exception as e:
        print(f"❌ 테스트 실행 오류: {e}")
        print("💡 GPU 메모리 부족일 수 있습니다. 런타임을 재시작해보세요.")

else:
    print("❌ 참조 이미지가 없어 테스트를 건너뜁니다.")
    print("💡 Google Drive에 test.png 파일을 업로드하고 다시 실행하세요.")

print("\n🎯 테스트 완료!")
print("\n💡 전체 시스템을 사용하려면 다음 셀의 find_similar_faces_auto_enhanced() 함수를 사용하세요.")

## 📚 Google Colab 사용 가이드

### ? Colab 환경 최적화

**이 노트북은 Google Colab에서 `face_recognition` 라이브러리 문제를 해결하기 위해 OpenCV 전용으로 설계되었습니다.**

### 🎯 주요 변경사항

1. **face_recognition 라이브러리 제거**: dlib 컴파일 오류 해결
2. **OpenCV 전용 얼굴 감지**: Haar Cascade + 강화된 특징 추출
3. **Colab 메모리 최적화**: CPU offload + attention slicing
4. **자동 환경 감지**: Colab과 로컬 환경 자동 구분

### 🚀 실행 순서

1. **셀 1**: 환경 설정 및 라이브러리 로드
2. **셀 2**: Stable Diffusion 파이프라인 초기화 (첫 실행 시 3-5분 소요)
3. **셀 3**: OpenCV 기반 얼굴 유사도 평가기
4. **셀 4-6**: 유틸리티 함수들
5. **셀 7**: 참조 이미지 확인
6. **셀 8**: 테스트 실행

### 📁 파일 경로 설정

```python
# Google Drive에 파일 업로드 경로:
# /content/drive/MyDrive/homepage/스프린트미션/스프린트미션_작업중/test.png

# 또는 Colab에 직접 업로드:
from google.colab import files
uploaded = files.upload()  # test.png 업로드
reference_photo = "test.png"
```

### ⚙️ 매개변수 권장값 (Colab 최적화)

- **`num_inference_steps`**: 20-30 (빠른 생성)
- **`target_score`**: 0.4-0.6 (OpenCV 특성상 낮게 설정)
- **`max_attempts`**: 20-30 (Colab 시간 제한)
- **`target_count`**: 3-5개

### 🔧 메모리 최적화 팁

```python
# GPU 메모리 부족 시:
import gc
import torch

# 메모리 정리
gc.collect()
torch.cuda.empty_cache()

# 파이프라인 재초기화
pipe.enable_model_cpu_offload()
```

### 🚨 Colab 주의사항

1. **런타임 연결 끊김**: 장시간 실행 시 연결이 끊길 수 있음
2. **GPU 사용량 제한**: 무료 계정은 GPU 사용량 제한 있음
3. **파일 저장**: 생성된 이미지는 Google Drive에 자동 저장
4. **메모리 제한**: 복잡한 작업 시 메모리 부족 발생 가능

### 💡 문제 해결

**"CUDA out of memory" 오류 시:**
```python
# 1. 런타임 재시작: 런타임 > 런타임 재시작
# 2. 메모리 정리 후 재실행
# 3. inference_steps 줄이기 (20으로 설정)
```

**참조 이미지를 찾을 수 없을 때:**
```python
# Google Drive 마운트 확인
from google.colab import drive
drive.mount('/content/drive')

# 파일 경로 확인
import os
print(os.listdir('/content/drive/MyDrive'))
```

In [ ]:
# 시스템 테스트 및 참조 이미지 확인
test_image_path = "test.png"

print("🧪 OpenCV 기반 시스템 테스트...")

# 참조 이미지 존재 확인
if os.path.exists(test_image_path):
    print(f"✅ 참조 이미지 발견: {test_image_path}")

    # OpenCV 기반 평가기 테스트
    try:
        evaluator = FaceSimilarityEvaluator(test_image_path)
        print("✅ OpenCV 기반 얼굴 평가기 초기화 성공!")

        # 자기 자신과의 유사도 테스트
        if evaluator.reference_features is not None:
            test_image = Image.open(test_image_path)
            scores = evaluator.evaluate_similarity(test_image)
            print(f"🔍 자기 참조 테스트 결과:")
            for key, value in scores.items():
                print(f"   {key}: {value:.4f}")
        else:
            print("⚠️ 참조 이미지에서 얼굴을 감지하지 못했습니다.")

    except Exception as e:
        print(f"❌ 평가기 초기화 실패: {e}")

else:
    print(f"⚠️ 참조 이미지가 없습니다: {test_image_path}")
    print("현재 디렉토리의 파일들:")
    for file in os.listdir("."):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"  📁 {file}")

print("🔧 시스템이 face_recognition 없이 정상 작동합니다!")

In [ ]:
# face_recognition 없이 작동하는 SmartPromptGenerator
class SmartPromptGenerator:
    def __init__(self, reference_image_path):
        """OpenCV 기반 스마트 프롬프트 생성기"""
        print(f"🎨 OpenCV 기반 프롬프트 생성기 초기화: {reference_image_path}")

        self.reference_image_path = reference_image_path
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        if os.path.exists(reference_image_path):
            self.reference_image = cv2.imread(reference_image_path)
            self.reference_image_rgb = cv2.cvtColor(self.reference_image, cv2.COLOR_BGR2RGB)
            self.features = self.analyze_features()
        else:
            print(f"❌ 참조 이미지를 찾을 수 없습니다: {reference_image_path}")
            self.features = {}

    def analyze_features(self):
        """OpenCV를 사용한 기본적인 얼굴 특징 분석"""
        features = {}

        try:
            # 얼굴 감지
            gray = cv2.cvtColor(self.reference_image, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

            if len(faces) > 0:
                # 가장 큰 얼굴 선택
                x, y, w, h = max(faces, key=lambda f: f[2] * f[3])

                # 기본 특징들
                features.update({
                    'face_detected': True,
                    'face_size': 'large' if w * h > 10000 else 'medium' if w * h > 5000 else 'small',
                    'face_ratio': w / h,
                })

                # 이미지 밝기 분석
                brightness = np.mean(gray[y:y+h, x:x+w])
                features['brightness'] = 'bright' if brightness > 150 else 'dark' if brightness < 100 else 'normal'

                # 색상 분석 (간단한 버전)
                face_roi = self.reference_image_rgb[y:y+h, x:x+w]
                avg_color = np.mean(face_roi, axis=(0, 1))
                features['skin_tone'] = 'warm' if avg_color[0] > avg_color[2] else 'cool'

            else:
                features['face_detected'] = False

        except Exception as e:
            print(f"특징 분석 오류: {e}")
            features['face_detected'] = False

        return features

    def generate_smart_prompts(self, count=10):
        """분석된 특징을 바탕으로 스마트 프롬프트 생성"""

        base_prompts = [
            "a professional portrait photo of a person",
            "a high-quality headshot of a person",
            "a realistic portrait of a person",
            "a detailed face portrait of a person",
            "a studio portrait of a person",
            "a professional photograph of a person",
            "a clear portrait photo of a person",
            "a high-resolution portrait of a person",
            "a realistic headshot of a person",
            "a professional studio portrait of a person"
        ]

        # 특징 기반 수정자들
        modifiers = []

        if self.features.get('face_detected', False):
            if self.features.get('brightness') == 'bright':
                modifiers.append("well-lit")
            elif self.features.get('brightness') == 'dark':
                modifiers.append("dramatic lighting")

            if self.features.get('skin_tone') == 'warm':
                modifiers.append("warm tones")
            elif self.features.get('skin_tone') == 'cool':
                modifiers.append("cool tones")

        # 품질 향상 키워드
        quality_keywords = [
            "high quality", "detailed", "sharp focus", "professional",
            "photorealistic", "8k resolution", "studio lighting"
        ]

        smart_prompts = []
        for i, base in enumerate(base_prompts[:count]):
            prompt = base

            # 수정자 추가
            if modifiers and i % 3 == 0:
                prompt += f", {', '.join(modifiers[:2])}"

            # 품질 키워드 추가
            if i % 2 == 0:
                prompt += f", {quality_keywords[i % len(quality_keywords)]}"

            smart_prompts.append(prompt)

        return smart_prompts, self.features

print("✅ OpenCV 기반 SmartPromptGenerator 클래스 로드 완료")

# 테스트
try:
    prompt_gen = SmartPromptGenerator("test.png")
    smart_prompts, features = prompt_gen.generate_smart_prompts(5)
    print("🎨 생성된 스마트 프롬프트:")
    for i, prompt in enumerate(smart_prompts, 1):
        print(f"  {i}. {prompt}")
    print(f"📊 분석된 특징: {features}")
except Exception as e:
    print(f"❌ 프롬프트 생성기 테스트 실패: {e}")

In [ ]:
# 개선된 find_similar_faces_auto_enhanced 함수 (나이, 성별, 인종 정보 추가)
import random
import matplotlib.pyplot as plt

def find_similar_faces_auto_enhanced(pipe, reference_image_path, target_score=0.6, max_attempts=50, target_count=3,
                                   age_group="20대", gender="여성", ethnicity="아시안"):
    """OpenCV 기반 자동 얼굴 유사도 탐색 (개인정보 기반 프롬프트 개선)"""

    print("🔄 개선된 OpenCV 기반 자동 유사도 탐색 시작...")
    print(f"🎯 검색 조건: {age_group} {gender} {ethnicity}")

    # OpenCV 기반 평가기 초기화
    try:
        print("🎯 OpenCV 기반 평가기 초기화...")
        evaluator = FaceSimilarityEvaluator(reference_image_path)
        prompt_gen = SmartPromptGenerator(reference_image_path)
        print("✅ OpenCV 기반 시스템 초기화 성공")

        # 기본 스마트 프롬프트 생성
        smart_prompts, features = prompt_gen.generate_smart_prompts()
        print(f"🎨 기본 {len(smart_prompts)}개의 스마트 프롬프트 생성 완료")

    except Exception as e:
        print(f"❌ OpenCV 기반 평가기 초기화 실패: {e}")
        return []

    # 개인정보 기반 향상된 프롬프트 생성
    enhanced_prompts = generate_demographic_prompts(age_group, gender, ethnicity)

    # 기본 프롬프트와 개인정보 기반 프롬프트 결합
    all_prompts = enhanced_prompts + smart_prompts[:5]  # 기본 프롬프트도 일부 포함

    print(f"🎨 총 {len(all_prompts)}개의 향상된 프롬프트 준비 완료")

    successful_images = []
    attempts = 0
    prompt_idx = 0

    print(f"🎯 목표: {target_count}개의 유사한 얼굴 찾기 (유사도 >= {target_score})")
    print(f"📊 분석된 특징: {features}")

    while len(successful_images) < target_count and attempts < max_attempts:
        attempts += 1
        current_prompt = all_prompts[prompt_idx % len(all_prompts)]

        print(f"\n🔄 시도 {attempts}/{max_attempts}")
        print(f"📝 프롬프트: {current_prompt}")

        try:
            # 이미지 생성
            with torch.no_grad():
                result = pipe(
                    current_prompt,
                    num_inference_steps=20,
                    guidance_scale=7.5,
                    width=512,
                    height=512
                )

            generated_image = result.images[0]

            # 유사도 평가
            scores = evaluator.evaluate_similarity(generated_image)

            print(f"📊 평가 결과:")
            for key, value in scores.items():
                print(f"   {key}: {value:.4f}")

            # 목표 점수 달성 시 저장
            if scores['total_score'] >= target_score:
                successful_images.append({
                    'image': generated_image,
                    'scores': scores,
                    'prompt': current_prompt,
                    'attempt': attempts,
                    'demographics': f"{age_group}_{gender}_{ethnicity}"
                })
                print(f"✅ 성공! ({len(successful_images)}/{target_count})")

                # 이미지 표시
                plt.figure(figsize=(6, 6))
                plt.imshow(generated_image)
                plt.title(f"Found #{len(successful_images)} (Score: {scores['total_score']:.3f})")
                plt.axis('off')
                plt.show()

            else:
                print(f"❌ 목표 점수 미달 (현재: {scores['total_score']:.4f}, 목표: {target_score})")

            prompt_idx += 1

        except Exception as e:
            print(f"❌ 이미지 생성 오류: {e}")
            prompt_idx += 1

    print(f"\n🎯 탐색 완료: {len(successful_images)}개의 성공적인 이미지 생성")

    # 결과 정렬 (총 점수 기준)
    successful_images.sort(key=lambda x: x['scores']['total_score'], reverse=True)

    return successful_images


def generate_demographic_prompts(age_group, gender, ethnicity):
    """나이, 성별, 인종 정보를 기반으로 향상된 프롬프트 생성"""

    # 나이 그룹별 키워드
    age_keywords = {
        "10대": ["young teenager", "youthful", "teenage", "adolescent"],
        "20대": ["young adult", "early twenties", "youthful adult", "young person"],
        "30대": ["adult", "mature young adult", "thirty-something", "professional adult"],
        "40대": ["middle-aged adult", "mature", "experienced adult", "forty-something"],
        "50대": ["mature adult", "middle-aged", "distinguished", "experienced person"]
    }

    # 성별 키워드
    gender_keywords = {
        "남성": ["man", "male", "gentleman", "guy"],
        "여성": ["woman", "female", "lady", "girl"]
    }

    # 인종 키워드
    ethnicity_keywords = {
        "아시안": ["Asian", "East Asian", "Korean", "Japanese", "Chinese"],
        "백인": ["Caucasian", "white", "European", "Western"],
        "흑인": ["African American", "black", "African", "dark-skinned"],
        "히스패닉": ["Hispanic", "Latino", "Latin American", "Mediterranean"],
        "중동": ["Middle Eastern", "Arabic", "Persian", "Turkish"]
    }

    # 기본 스타일 (밝고 자연스러운 톤)
    lighting_styles = [
        "natural lighting, bright and cheerful",
        "soft natural daylight, warm atmosphere",
        "bright studio lighting, clean background",
        "outdoor natural light, fresh and bright",
        "golden hour lighting, warm and inviting",
        "soft window light, natural and bright"
    ]

    # 품질 향상 키워드
    quality_keywords = [
        "high quality, detailed, photorealistic",
        "professional photography, sharp focus, 8k resolution",
        "natural skin tone, realistic features, detailed",
        "clear facial features, professional portrait",
        "realistic lighting, natural expression, high detail"
    ]

    enhanced_prompts = []

    # 각 조합으로 프롬프트 생성
    age_terms = age_keywords.get(age_group, ["young adult"])
    gender_terms = gender_keywords.get(gender, ["person"])
    ethnic_terms = ethnicity_keywords.get(ethnicity, [""])

    for age_term in age_terms[:2]:  # 상위 2개 나이 키워드
        for gender_term in gender_terms[:2]:  # 상위 2개 성별 키워드
            for ethnic_term in ethnic_terms[:2]:  # 상위 2개 인종 키워드
                for lighting in lighting_styles[:3]:  # 상위 3개 조명 스타일
                    for quality in quality_keywords[:2]:  # 상위 2개 품질 키워드

                        # 프롬프트 조합
                        if ethnic_term:
                            prompt = f"professional portrait of a {age_term} {ethnic_term} {gender_term}, {lighting}, {quality}"
                        else:
                            prompt = f"professional portrait of a {age_term} {gender_term}, {lighting}, {quality}"

                        enhanced_prompts.append(prompt)

    # 중복 제거 및 셔플
    enhanced_prompts = list(set(enhanced_prompts))
    random.shuffle(enhanced_prompts)

    print(f"🎨 {age_group} {gender} {ethnicity} 기반 {len(enhanced_prompts)}개 프롬프트 생성")

    return enhanced_prompts[:15]  # 상위 15개만 반환


print("✅ 개선된 find_similar_faces_auto_enhanced 함수 로드 완료")
print("🚀 이제 나이, 성별, 인종 정보로 더 정확한 검색이 가능합니다!")

In [ ]:
# 개선된 함수 사용 예제
print("=" * 60)
print("🎯 개선된 find_similar_faces_auto_enhanced 사용 예제")
print("=" * 60)

# 사용 가능한 옵션들
print("📋 사용 가능한 옵션들:")
print("나이 그룹: 10대, 20대, 30대, 40대, 50대")
print("성별: 남성, 여성")
print("인종: 아시안, 백인, 흑인, 히스패닉, 중동")
print()

# 예제 프롬프트 생성 테스트
print("🧪 예제 프롬프트 생성 테스트:")
test_prompts = generate_demographic_prompts("20대", "여성", "아시안")
print("생성된 프롬프트 예시 (상위 5개):")
for i, prompt in enumerate(test_prompts[:5], 1):
    print(f"  {i}. {prompt}")

print("\n💡 사용법:")
print("""
# 기본 사용 (20대 여성 아시안)
similar_faces = find_similar_faces_auto_enhanced(
    pipe=pipe,
    reference_image_path="test.png",
    target_score=0.6,
    max_attempts=30,
    target_count=3
)

# 커스텀 설정 (30대 남성 백인)
similar_faces = find_similar_faces_auto_enhanced(
    pipe=pipe,
    reference_image_path="test.png",
    target_score=0.65,
    max_attempts=50,
    target_count=5,
    age_group="30대",
    gender="남성",
    ethnicity="백인"
)
""")

In [ ]:
# !pip install -q "numpy<2"
# !pip install -q kaggle
# !pip install -q kagglehub
# !pip install -q albumentations
# !pip install -q ultralytics
# !pip install -q --user opencv-python
# !pip install -q torchvision
# !pip install -q torch
# !pip install -q pycocotools
#!pip install --upgrade torchvision
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

!pip install -q lpips
print(f"lpips 설치 완료")
!pip install -q pytorch-fid
print(f"pytorch-fid 설치 완료")
!pip install -q torch-fidelity
print(f"torch-fidelity 설치 완료")
!pip install -q scipy
print(f"scipy 설치 완료")
!pip install -q face_recognition
print(f"face_recognition 설치 완료")
!pip install -q diffusers
print(f"diffusers 설치 완료")

print(f"라이브러리 설치 완료")

In [ ]:
# 대안 방법: OpenCV 기반 얼굴 유사도 평가기
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import lpips
import torchvision.transforms as transforms

class OpenCVFaceSimilarityEvaluator:
    def __init__(self, reference_image_path):
        """OpenCV 기반 얼굴 유사도 평가기"""
        print(f"OpenCV 기반 평가기 초기화 중: {reference_image_path}")

        # OpenCV 얼굴 감지기 초기화
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # 참조 이미지 로드
        if os.path.exists(reference_image_path):
            self.reference_image = cv2.imread(reference_image_path)
            self.reference_image_rgb = cv2.cvtColor(self.reference_image, cv2.COLOR_BGR2RGB)

            # 참조 얼굴 특징 추출
            self.reference_features = self.extract_face_features(self.reference_image)

            if self.reference_features is not None:
                print("✅ 참조 얼굴 특징 추출 완료")
            else:
                print("⚠️ 참조 이미지에서 얼굴을 찾을 수 없습니다")

        else:
            raise FileNotFoundError(f"참조 이미지를 찾을 수 없습니다: {reference_image_path}")

        # LPIPS 초기화
        try:
            self.lpips_model = lpips.LPIPS(net='alex').to(__device)
            self.transform = transforms.Compose([
                transforms.Resize((512, 512)),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])
            print("✅ LPIPS 모델 초기화 완료")
        except Exception as e:
            print(f"⚠️ LPIPS 초기화 실패: {e}")
            self.lpips_model = None

    def extract_face_features(self, image):
        """얼굴 특징 추출 (OpenCV 기반)"""
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

            if len(faces) == 0:
                return None

            # 가장 큰 얼굴 선택
            largest_face = max(faces, key=lambda x: x[2] * x[3])
            x, y, w, h = largest_face

            # 얼굴 영역 추출
            face_roi = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (128, 128))

            # 히스토그램 기반 특징
            hist_features = cv2.calcHist([face_resized], [0], None, [256], [0, 256]).flatten()

            # LBP (Local Binary Pattern) 특징
            lbp_features = self.calculate_lbp(face_resized)

            # 특징 결합
            combined_features = np.concatenate([hist_features, lbp_features])

            return combined_features

        except Exception as e:
            print(f"특징 추출 오류: {e}")
            return None

    def calculate_lbp(self, image, radius=3, n_points=24):
        """Local Binary Pattern 계산"""
        try:
            from skimage.feature import local_binary_pattern
            lbp = local_binary_pattern(image, n_points, radius, method='uniform')
            (hist, _) = np.histogram(lbp.ravel(), bins=n_points + 2, range=(0, n_points + 2))
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-7)
            return hist
        except:
            # skimage가 없으면 간단한 대안 사용
            return np.histogram(image.ravel(), bins=50)[0].astype("float")

    def evaluate_similarity(self, generated_image):
        """생성된 이미지와 참조 이미지의 유사도 평가"""
        scores = {}

        try:
            # PIL을 OpenCV 형식으로 변환
            generated_array = np.array(generated_image)
            generated_cv = cv2.cvtColor(generated_array, cv2.COLOR_RGB2BGR)

            # 1. OpenCV 기반 얼굴 유사도
            generated_features = self.extract_face_features(generated_cv)

            if generated_features is not None and self.reference_features is not None:
                # 코사인 유사도 계산
                similarity = cosine_similarity([self.reference_features], [generated_features])[0][0]
                face_similarity = max(0, similarity)  # 음수 방지
                scores['face_similarity'] = face_similarity
            else:
                scores['face_similarity'] = 0.0

            # 2. LPIPS 점수 (가능한 경우)
            if self.lpips_model is not None:
                try:
                    ref_tensor = self.transform(Image.fromarray(self.reference_image_rgb)).unsqueeze(0).to(__device)
                    gen_tensor = self.transform(generated_image).unsqueeze(0).to(__device)

                    with torch.no_grad():
                        lpips_distance = self.lpips_model(ref_tensor, gen_tensor).item()
                        lpips_similarity = max(0, 1 - lpips_distance)
                        scores['lpips_similarity'] = lpips_similarity
                except Exception as lpips_e:
                    print(f"LPIPS 계산 오류: {lpips_e}")
                    scores['lpips_similarity'] = 0.0
            else:
                scores['lpips_similarity'] = 0.0

            # 3. 종합 점수
            face_weight = 0.8  # OpenCV 기반이므로 가중치 조정
            lpips_weight = 0.2

            total_score = (scores['face_similarity'] * face_weight +
                          scores['lpips_similarity'] * lpips_weight)
            scores['total_score'] = total_score

        except Exception as e:
            print(f"평가 오류: {e}")
            scores = {'face_similarity': 0.0, 'lpips_similarity': 0.0, 'total_score': 0.0}

        return scores

print("OpenCV 기반 얼굴 평가기 로드 완료")

In [ ]:
# 기존 find_similar_faces_auto 함수를 OpenCV 기반으로 업그레이드
def find_similar_faces_auto_enhanced(pipe, reference_image_path, target_score=0.6, max_attempts=50, target_count=3, use_opencv_fallback=True):
    """향상된 자동 얼굴 유사도 탐색 (face_recognition + OpenCV 백업)"""

    print("🔄 자동 유사도 탐색 시작...")

    # 먼저 face_recognition 시도
    evaluator = None
    prompt_gen = None

    try:
        print("🎯 face_recognition 기반 평가기 시도...")
        evaluator = FaceSimilarityEvaluator(reference_image_path)
        prompt_gen = SmartPromptGenerator(reference_image_path)
        print("✅ face_recognition 기반 시스템 초기화 성공")

        # 스마트 프롬프트 생성
        smart_prompts, features = prompt_gen.generate_smart_prompts()
        print(f"감지된 얼굴 특징: {features}")

    except Exception as face_rec_error:
        print(f"❌ face_recognition 초기화 실패: {face_rec_error}")

        if use_opencv_fallback:
            print("🔄 OpenCV 기반 백업 시스템으로 전환...")
            try:
                evaluator = OpenCVFaceSimilarityEvaluator(reference_image_path)
                print("✅ OpenCV 기반 평가기 초기화 성공")

                # 기본 프롬프트 사용
                smart_prompts = [
                    "professional portrait, high quality, detailed, sharp focus",
                    "natural lighting portrait, realistic, professional photography",
                    "soft studio lighting, headshot, detailed face",
                    "outdoor natural portrait, clear lighting, high resolution",
                    "casual portrait, friendly expression, detailed features",
                    "business portrait, professional attire, studio lighting"
                ]
                features = {"system": "opencv_fallback"}

            except Exception as opencv_error:
                print(f"❌ OpenCV 백업 시스템도 실패: {opencv_error}")
                return []
        else:
            return []

    if not evaluator:
        print("❌ 평가 시스템을 초기화할 수 없습니다.")
        return []

    # 공통 탐색 로직
    found_images = []
    attempt_count = 0

    # 네거티브 프롬프트
    negative_prompt = "blurry, low quality, bad anatomy, deformed, cartoon, anime, multiple faces, bad hands, distorted face"

    print(f"🎯 목표: {target_count}개 이미지 (점수 {target_score} 이상)")
    print("=" * 50)

    while len(found_images) < target_count and attempt_count < max_attempts:
        attempt_count += 1

        # 프롬프트 선택 (순환)
        prompt_idx = (attempt_count - 1) % len(smart_prompts)
        current_prompt = smart_prompts[prompt_idx]

        # 랜덤 시드
        seed = random.randint(1, 99999999)

        print(f"\n🔍 시도 {attempt_count}/{max_attempts}")
        print(f"📝 프롬프트: {current_prompt}")
        print(f"🎲 시드: {seed}")

        try:
            # 이미지 생성 (기존 함수 재사용)
            generated_image = alternative_face_generation(
                pipe=pipe,
                prompt=current_prompt,
                negative_prompt=negative_prompt,
                seed=seed,
                show_image=False  # 배치 처리시 화면 출력 끄기
            )

            if generated_image is None:
                print("❌ 이미지 생성 실패")
                continue

            # 유사도 평가
            scores = evaluator.evaluate_similarity(generated_image)
            total_score = scores['total_score']

            print(f"📊 점수: {total_score:.3f} (얼굴: {scores['face_similarity']:.3f}, LPIPS: {scores['lpips_similarity']:.3f})")

            # 목표 점수 달성시 저장
            if total_score >= target_score:
                print(f"✅ 목표 달성! (점수: {total_score:.3f})")

                # 이미지 저장
                save_path = f"similar_face_{len(found_images)+1:02d}_score_{total_score:.3f}_seed_{seed}.png"
                generated_image.save(save_path)

                found_images.append({
                    'image': generated_image,
                    'scores': scores,
                    'prompt': current_prompt,
                    'seed': seed,
                    'path': save_path
                })

                # 이미지 표시
                plt.figure(figsize=(6, 6))
                plt.imshow(generated_image)
                plt.title(f"Found #{len(found_images)} (Score: {total_score:.3f})")
                plt.axis('off')
                plt.show()

        except Exception as e:
            print(f"❌ 시도 {attempt_count} 오류: {e}")
            continue

    print("\n" + "=" * 50)
    print(f"🏁 탐색 완료!")
    print(f"🔢 시도 횟수: {attempt_count}")
    print(f"🖼️ 발견된 이미지: {len(found_images)}개")

    if found_images:
        print("\n📋 발견된 이미지 정보:")
        for i, result in enumerate(found_images):
            print(f"  {i+1}. 점수: {result['scores']['total_score']:.3f}, 경로: {result['path']}")

    return found_images

print("✅ 통합 자동 탐색 함수 로드 완료")

In [ ]:
# 🚀 통합 테스트 실행 (기존 함수 재사용)
print("🔧 환경 감지 및 설정...")

# 경로 설정 (helper 함수 재사용)
import os

# 코랩 여부 확인 (helper 모듈 사용)
try:
    is_colab = helper.is_colab
except:
    # helper가 없으면 직접 확인
    is_colab = 'google.colab' in str(get_ipython())

if is_colab:
    colab_path = "/content/drive/MyDrive/homepage/스프린트미션/스프린트미션_작업중/test.png"
    reference_photo = colab_path
    print(f"✅ 코랩 환경 - 경로: {reference_photo}")
else:
    local_path = "test.png"
    reference_photo = local_path
    print(f"✅ 로컬 환경 - 경로: {reference_photo}")

# 파일 존재 확인
if not os.path.exists(reference_photo):
    print("❌ 참조 이미지를 찾을 수 없습니다")
    print("현재 디렉토리:", os.getcwd())
    print("이미지 파일 목록:")
    for file in os.listdir('.'):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"  - {file}")
    reference_photo = None

if reference_photo:
    print(f"\n🎯 통합 자동 탐색 실행...")

    try:
        # 기존 함수 재사용 (enhanced 버전)
        similar_faces = find_similar_faces_auto_enhanced(
            pipe=pipe,
            reference_image_path=reference_photo,
            target_score=0.5,        # 관대한 점수
            max_attempts=20,         # 테스트용 적은 시도
            target_count=2,          # 2개만 찾기
            age_group="30대",
            gender="남성",
            ethnicity="백인",
            use_opencv_fallback=True # OpenCV 백업 활성화
        )

        if similar_faces:
            print(f"\n🎉 성공! {len(similar_faces)}개의 유사한 이미지를 찾았습니다!")

            # 히스토리 관리자 사용 (기존 gen_manager 재사용)
            print("\n📚 생성 히스토리 저장 중...")
            for result in similar_faces:
                metadata = {
                    "prompt": result['prompt'],
                    "seed": result['seed'],
                    "scores": result['scores'],
                    "method": "auto_search",
                    "model": "runwayml/stable-diffusion-v1-5"
                }
                gen_id = gen_manager.save_generation(metadata, result['path'])
                print(f"  히스토리 ID {gen_id}: {result['path']}")

        else:
            print("\n😢 조건에 맞는 이미지를 찾지 못했습니다.")
            print("💡 점수를 더 낮추거나 시도 횟수를 늘려보세요.")

    except Exception as e:
        print(f"❌ 통합 탐색 실패: {e}")

        print("\n🔄 기본 이미지 생성 테스트 (기존 함수 재사용)...")
        try:
            # enhanced_face_generation 함수 재사용
            test_image = enhanced_face_generation(
                pipe=pipe,
                prompt="professional portrait, high quality, realistic, detailed face",
                negative_prompt="blurry, low quality, bad anatomy, deformed",
                seed=42,
                save_path="fallback_test.png"
            )

            if test_image:
                print("✅ 기본 이미지 생성 성공! 시스템이 정상 작동합니다.")
        except Exception as fallback_e:
            print(f"❌ 기본 생성도 실패: {fallback_e}")

else:
    print("❌ 참조 이미지가 없어 테스트를 실행할 수 없습니다.")
    print("💡 test.png 파일을 현재 디렉토리에 업로드해주세요.")

In [ ]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

In [ ]:
# 기본 라이브러리

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import average_precision_score

# --- 기타 라이브러리 ---
import cv2
from PIL import Image
from PIL import ImageFilter
from PIL import ImageDraw
import albumentations as A
import IPython.display
#from tqdm import tqdm
from tqdm.notebook import tqdm

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as TF
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from collections import OrderedDict
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 기타 ---
import re
import os
import sys
import copy
import json
import math
import random
import yaml
import shutil
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from datetime import datetime
from datetime import timezone, timedelta
import pytz
__kst = pytz.timezone('Asia/Seoul')

# GPU 설정
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

  # 재현 가능한 결과를 위해
np.random.seed(42)
torch.manual_seed(42)
if __device == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치:{__device}")

In [ ]:
!pip install -q torch torchvision matplotlib pillow requests
import requests
import io
from diffusers import StableDiffusionPipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
pipe = pipe.to(device)

In [ ]:
def alternative_face_generation(pipe, prompt, negative_prompt=None, seed=42,
                              num_inference_steps=30, guidance_scale=7.5,
                              save_path=None, show_image=True):
    """확장된 Stable Diffusion 이미지 생성 함수"""

    try:
        # 기본 negative prompt 설정
        if negative_prompt is None:
            negative_prompt = "blurry, low quality, bad anatomy, deformed, extra limbs, bad hands, bad face, bad eyes, extra fingers, cartoon, anime, nsfw, inappropriate"

        # 랜덤 시드 고정
        generator = torch.Generator(device=pipe.device).manual_seed(seed)

        print(f"이미지 생성 중... (시드: {seed})")
        print(f"프롬프트: {prompt}")
        if negative_prompt:
            print(f"네거티브: {negative_prompt}")

        # 이미지 생성
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=generator,
            height=512,
            width=512
        ).images[0]

        # 이미지 저장
        if save_path:
            image.save(save_path)
            print(f"이미지 저장됨: {save_path}")

            # 메타데이터 저장
            metadata = {
                "prompt": prompt,
                "negative_prompt": negative_prompt,
                "seed": seed,
                "steps": num_inference_steps,
                "guidance_scale": guidance_scale
            }

            import json
            metadata_path = save_path.replace('.png', '_metadata.json')
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)

        # 이미지 표시
        if show_image:
            plt.figure(figsize=(4, 4))
            plt.imshow(image)
            plt.title(f"Generated (Seed: {seed})", fontsize=14)
            plt.axis('off')
            plt.show()

        print("이미지 생성 완료!")
        return image

    except Exception as e:
        print(f"생성 실패: {e}")
        return None

In [ ]:
# 🎯 OpenCV 기반 얼굴 유사도 평가기 (face_recognition 완전 대체)
print("🔄 OpenCV 전용 얼굴 분석 시스템 초기화...")

import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

try:
    import lpips
    LPIPS_AVAILABLE = True
    print("✅ LPIPS 사용 가능")
except ImportError:
    LPIPS_AVAILABLE = False
    print("⚠️ LPIPS 없음 - 기본 특징만 사용")

class OpenCVFaceEvaluator:
    """face_recognition 없이 OpenCV만으로 얼굴 유사도 평가"""

    def __init__(self, reference_image_path):
        print(f"🎯 OpenCV 얼굴 평가기 초기화: {reference_image_path}")

        # OpenCV 얼굴 감지기들 초기화
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

        # 참조 이미지 로드 및 특징 추출
        if not os.path.exists(reference_image_path):
            raise FileNotFoundError(f"참조 이미지를 찾을 수 없습니다: {reference_image_path}")

        self.reference_image = cv2.imread(reference_image_path)
        if self.reference_image is None:
            raise ValueError(f"이미지 로드 실패: {reference_image_path}")

        self.reference_image_rgb = cv2.cvtColor(self.reference_image, cv2.COLOR_BGR2RGB)
        self.reference_features = self.extract_face_features(self.reference_image)

        if self.reference_features is None:
            raise ValueError("참조 이미지에서 얼굴을 찾을 수 없습니다")

        print("✅ 참조 이미지 특징 추출 완료")

        # LPIPS 초기화 (사용 가능한 경우)
        if LPIPS_AVAILABLE:
            try:
                self.lpips_model = lpips.LPIPS(net='alex').to(device)
                self.transform = transforms.Compose([
                    transforms.Resize((256, 256)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                ])
                print("✅ LPIPS 모델 초기화 완료")
            except Exception as e:
                print(f"⚠️ LPIPS 초기화 실패: {e}")
                self.lpips_model = None
        else:
            self.lpips_model = None

    def extract_face_features(self, image):
        """OpenCV 기반 종합 얼굴 특징 추출"""
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # 다중 스케일 얼굴 감지
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(50, 50))

            if len(faces) == 0:
                return None

            # 가장 큰 얼굴 선택
            x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
            face_roi = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (128, 128))
            face_normalized = cv2.equalizeHist(face_resized)

            # 1. 히스토그램 특징
            hist_features = cv2.calcHist([face_normalized], [0], None, [64], [0, 256]).flatten()
            hist_features = hist_features / (hist_features.sum() + 1e-7)

            # 2. LBP 특징 (간단한 구현)
            lbp_features = self.calculate_simple_lbp(face_normalized)

            # 3. 기하학적 특징
            geometric_features = np.array([
                w / h,  # 얼굴 비율
                w, h,   # 크기
                len(self.eye_cascade.detectMultiScale(face_roi)),  # 눈 개수
                np.std(face_normalized),  # 텍스처 변화도
                np.mean(face_normalized)  # 평균 밝기
            ])

            # 4. 에지 특징
            edges = cv2.Canny(face_normalized, 50, 150)
            edge_features = np.array([
                np.sum(edges > 0) / (128 * 128),  # 에지 밀도
                np.mean(edges),
                np.std(edges)
            ])

            # 모든 특징 결합
            all_features = np.concatenate([
                hist_features,      # 64개
                lbp_features,       # 26개
                geometric_features, # 6개
                edge_features      # 3개
            ])

            return all_features

        except Exception as e:
            print(f"특징 추출 오류: {e}")
            return None

    def calculate_simple_lbp(self, image):
        """간단한 LBP 계산"""
        h, w = image.shape
        lbp = np.zeros((h-2, w-2), dtype=np.uint8)

        for i in range(1, h-1):
            for j in range(1, w-1):
                center = image[i, j]
                binary_val = 0

                # 8방향 이웃 검사
                neighbors = [
                    image[i-1, j-1], image[i-1, j], image[i-1, j+1],
                    image[i, j+1], image[i+1, j+1], image[i+1, j],
                    image[i+1, j-1], image[i, j-1]
                ]

                for k, neighbor in enumerate(neighbors):
                    if neighbor >= center:
                        binary_val += 2**k

                lbp[i-1, j-1] = binary_val

        # 히스토그램 계산
        hist, _ = np.histogram(lbp.ravel(), bins=26, range=(0, 256))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        return hist

    def evaluate_similarity(self, generated_image):
        """생성된 이미지와의 유사도 평가"""
        scores = {}

        try:
            # PIL을 OpenCV 형식으로 변환
            generated_array = np.array(generated_image)
            generated_cv = cv2.cvtColor(generated_array, cv2.COLOR_RGB2BGR)

            # 얼굴 특징 추출
            generated_features = self.extract_face_features(generated_cv)

            if generated_features is not None and self.reference_features is not None:
                # 1. 코사인 유사도
                cosine_sim = cosine_similarity([self.reference_features], [generated_features])[0][0]
                scores['face_similarity'] = max(0, cosine_sim)

                # 2. 유클리드 거리 기반 유사도
                euclidean_dist = np.linalg.norm(self.reference_features - generated_features)
                euclidean_sim = 1 / (1 + euclidean_dist / 100)
                scores['euclidean_similarity'] = euclidean_sim
            else:
                scores['face_similarity'] = 0.0
                scores['euclidean_similarity'] = 0.0

            # 3. LPIPS (사용 가능한 경우)
            if self.lpips_model is not None:
                try:
                    ref_tensor = self.transform(Image.fromarray(self.reference_image_rgb)).unsqueeze(0).to(device)
                    gen_tensor = self.transform(generated_image).unsqueeze(0).to(device)

                    with torch.no_grad():
                        lpips_distance = self.lpips_model(ref_tensor, gen_tensor).item()
                        lpips_similarity = max(0, 1 - lpips_distance)
                        scores['lpips_similarity'] = lpips_similarity
                except Exception as lpips_e:
                    print(f"LPIPS 계산 오류: {lpips_e}")
                    scores['lpips_similarity'] = 0.0
            else:
                scores['lpips_similarity'] = 0.0

            # 4. 종합 점수
            face_weight = 0.5
            euclidean_weight = 0.3
            lpips_weight = 0.2

            total_score = (scores['face_similarity'] * face_weight +
                          scores['euclidean_similarity'] * euclidean_weight +
                          scores['lpips_similarity'] * lpips_weight)
            scores['total_score'] = total_score

        except Exception as e:
            print(f"평가 오류: {e}")
            scores = {
                'face_similarity': 0.0,
                'euclidean_similarity': 0.0,
                'lpips_similarity': 0.0,
                'total_score': 0.0
            }

        return scores

print("✅ OpenCV 기반 얼굴 평가기 클래스 정의 완료!")
print("🎯 이제 face_recognition 없이도 완전한 얼굴 유사도 분석이 가능합니다.")

# 사용 예제
print("\n💡 사용 예제:")
print("evaluator = OpenCVFaceEvaluator('your_reference_image.png')")
print("scores = evaluator.evaluate_similarity(generated_image)")
print("print(f'유사도: {scores[\"total_score\"]:.4f}')")

In [ ]:
class SmartPromptGenerator:
    def __init__(self, reference_image_path):
        """참조 이미지에서 특징 추출하여 프롬프트 생성"""
        print(f"프롬프트 생성기 초기화 중: {reference_image_path}")

        # 이미지가 존재하는지 확인
        if not os.path.exists(reference_image_path):
            raise FileNotFoundError(f"참조 이미지를 찾을 수 없습니다: {reference_image_path}")

        try:
            self.reference_image = face_recognition.load_image_file(reference_image_path)
            print("얼굴 랜드마크 감지 중...")
            self.face_landmarks = face_recognition.face_landmarks(self.reference_image)

            if self.face_landmarks:
                print(f"얼굴 랜드마크 감지 완료 ({len(self.face_landmarks)}개 얼굴)")
            else:
                print("얼굴 랜드마크를 찾을 수 없습니다. 기본 프롬프트를 사용합니다.")

        except Exception as e:
            print(f"프롬프트 생성기 초기화 오류: {e}")
            self.reference_image = None
            self.face_landmarks = []

    def analyze_face_features(self):
        """얼굴 특징 분석"""
        if not self.face_landmarks:
            print("얼굴 랜드마크가 없어 기본 특징을 반환합니다.")
            return {
                'eye_size': 'normal eyes',
                'nose_type': 'straight nose',
                'face_shape': 'balanced face'
            }

        try:
            landmarks = self.face_landmarks[0]
            features = {}

            # 눈 분석
            left_eye = landmarks.get('left_eye', [])
            right_eye = landmarks.get('right_eye', [])

            if left_eye and len(left_eye) >= 4:
                eye_width = abs(left_eye[3][0] - left_eye[0][0])
                features['eye_size'] = 'large eyes' if eye_width > 30 else 'normal eyes'
            else:
                features['eye_size'] = 'normal eyes'

            # 코 분석
            nose_bridge = landmarks.get('nose_bridge', [])
            features['nose_type'] = 'straight nose' if len(nose_bridge) > 3 else 'small nose'

            # 얼굴형 분석 (간단한 버전)
            chin = landmarks.get('chin', [])
            nose_bridge = landmarks.get('nose_bridge', [])

            if chin and len(chin) >= 9 and nose_bridge and len(nose_bridge) >= 1:
                face_width = abs(chin[0][0] - chin[-1][0])
                face_height = abs(chin[8][1] - nose_bridge[0][1])

                if face_height > face_width * 1.3:
                    features['face_shape'] = 'oval face'
                elif face_width > face_height * 1.1:
                    features['face_shape'] = 'round face'
                else:
                    features['face_shape'] = 'balanced face'
            else:
                features['face_shape'] = 'balanced face'

            return features

        except Exception as e:
            print(f"얼굴 특징 분석 오류: {e}")
            return {
                'eye_size': 'normal eyes',
                'nose_type': 'straight nose',
                'face_shape': 'balanced face'
            }

    def generate_smart_prompts(self, base_descriptions=None):
        """분석된 특징을 바탕으로 스마트 프롬프트 생성"""
        features = self.analyze_face_features()

        if base_descriptions is None:
            base_descriptions = [
                "professional portrait",
                "natural lighting portrait",
                "soft studio lighting",
                "outdoor natural portrait",
                "casual portrait",
                "headshot photography"
            ]

        # 특징 기반 수식어 추가
        feature_terms = []
        for feature_type, feature_value in features.items():
            feature_terms.append(feature_value)

        # 기본 품질 향상 키워드
        quality_terms = [
            "high quality", "detailed", "sharp focus",
            "professional photography", "realistic", "8k resolution"
        ]

        prompts = []
        for base_desc in base_descriptions:
            # 특징 + 기본 설명 + 품질 키워드 조합
            feature_str = ", ".join(feature_terms[:2])  # 너무 많으면 과적합
            quality_str = ", ".join(quality_terms[:3])

            prompt = f"{base_desc}, {feature_str}, {quality_str}"
            prompts.append(prompt)

        print(f"생성된 프롬프트 수: {len(prompts)}")
        return prompts, features

In [ ]:
class ImageGenerationManager:
    def __init__(self, history_file="generation_history.json"):
        """이미지 생성 히스토리 관리자"""
        self.history_file = history_file
        self.load_history()

    def load_history(self):
        """기존 히스토리 로드"""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r', encoding='utf-8') as f:
                self.history = json.load(f)
        else:
            self.history = {"generations": []}

    def save_generation(self, metadata, image_path=None):
        """생성 정보 저장"""
        generation_data = {
            **metadata,
            "timestamp": datetime.now().isoformat(),
            "image_path": image_path,
            "generation_id": len(self.history["generations"]) + 1
        }

        self.history["generations"].append(generation_data)

        # 히스토리 파일 저장
        with open(self.history_file, 'w', encoding='utf-8') as f:
            json.dump(self.history, f, indent=2, ensure_ascii=False)

        return generation_data["generation_id"]

    def regenerate_by_id(self, generation_id, pipe, save_new=True):
        """ID로 이미지 재생성"""
        generation = self.find_generation_by_id(generation_id)
        if not generation:
            print(f"ID {generation_id}를 찾을 수 없습니다.")
            return None

        return self.regenerate_from_metadata(generation, pipe, save_new)

    def regenerate_from_metadata(self, metadata, pipe, save_new=True):
        """메타데이터로부터 이미지 재생성"""
        print(f"재생성 시작...")
        print(f"원본 시드: {metadata.get('seed', 'Unknown')}")
        print(f"원본 프롬프트: {metadata.get('prompt', 'Unknown')}")

        # 새 파일명 생성 (재생성용)
        save_path = None
        if save_new:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            save_path = f"regenerated_{timestamp}_seed_{metadata.get('seed', 'unknown')}.png"

        # 이미지 재생성
        regenerated_image = alternative_face_generation(
            pipe=pipe,
            prompt=metadata.get('prompt', ''),
            negative_prompt=metadata.get('negative_prompt', ''),
            seed=metadata.get('seed', 42),
            num_inference_steps=metadata.get('steps', 30),
            guidance_scale=metadata.get('guidance_scale', 7.5),
            save_path=save_path,
            show_image=True
        )

        if regenerated_image and save_new:
            # 재생성 정보도 히스토리에 추가
            regen_metadata = {
                **metadata,
                "is_regeneration": True,
                "original_generation_id": metadata.get('generation_id'),
                "regenerated_from": metadata.get('image_path', 'Unknown')
            }
            self.save_generation(regen_metadata, save_path)

        return regenerated_image

    def find_generation_by_id(self, generation_id):
        """ID로 생성 정보 찾기"""
        for gen in self.history["generations"]:
            if gen.get("generation_id") == generation_id:
                return gen
        return None

    def search_generations(self, keyword=None, seed=None, limit=10):
        """생성 히스토리 검색"""
        results = []

        for gen in self.history["generations"]:
            match = True

            if keyword and keyword.lower() not in gen.get("prompt", "").lower():
                match = False

            if seed and gen.get("seed") != seed:
                match = False

            if match:
                results.append(gen)

        return results[-limit:]  # 최근 항목부터

    def show_history(self, limit=10):
        """히스토리 출력"""
        recent = self.history["generations"][-limit:]

        print(f"최근 {len(recent)}개 생성 기록:")
        print("=" * 80)

        for gen in reversed(recent):  # 최신순
            print(f"ID: {gen.get('generation_id', 'Unknown')}")
            print(f"시드: {gen.get('seed', 'Unknown')}")
            print(f"시간: {gen.get('timestamp', 'Unknown')}")
            print(f"프롬프트: {gen.get('prompt', 'Unknown')[:60]}...")
            if gen.get('image_path'):
                print(f"파일: {gen.get('image_path')}")
            print("-" * 40)

In [ ]:
# 전역 매니저 인스턴스
gen_manager = ImageGenerationManager()

# 기존 함수 개선 (메타데이터 자동 저장)
def enhanced_face_generation(pipe, prompt, negative_prompt=None, seed=42,
                           num_inference_steps=30, guidance_scale=7.5,
                           save_path=None, show_image=True, auto_save_history=True):
    """히스토리 저장 기능이 추가된 이미지 생성"""

    # 기존 생성 함수 호출
    image = alternative_face_generation(
        pipe=pipe,
        prompt=prompt,
        negative_prompt=negative_prompt,
        seed=seed,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        save_path=save_path,
        show_image=show_image
    )

    # 히스토리 자동 저장
    if image and auto_save_history:
        metadata = {
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "seed": seed,
            "steps": num_inference_steps,
            "guidance_scale": guidance_scale,
            "model": "runwayml/stable-diffusion-v1-5"
        }

        generation_id = gen_manager.save_generation(metadata, save_path)
        print(f"히스토리 저장됨 (ID: {generation_id})")

    return image

def find_similar_faces_auto(pipe, reference_image_path, target_score=0.7, max_attempts=100, target_count=3):
    """자동으로 유사한 얼굴 이미지 찾기"""

    # 평가기와 프롬프트 생성기 초기화
    evaluator = FaceSimilarityEvaluator(reference_image_path)
    prompt_gen = SmartPromptGenerator(reference_image_path)

    # 스마트 프롬프트 생성
    smart_prompts, features = prompt_gen.generate_smart_prompts()
    print(f"감지된 얼굴 특징: {features}")
    print(f"생성된 스마트 프롬프트 수: {len(smart_prompts)}")

    found_images = []
    attempt_count = 0

    # 네거티브 프롬프트
    negative_prompt = "blurry, low quality, bad anatomy, deformed, cartoon, anime, multiple faces, bad hands"

    print(f"목표: {target_count}개 이미지 (점수 {target_score} 이상)")
    print("=" * 50)

    while len(found_images) < target_count and attempt_count < max_attempts:
        attempt_count += 1

        # 프롬프트 선택 (순환 또는 랜덤)
        prompt_idx = (attempt_count - 1) % len(smart_prompts)
        current_prompt = smart_prompts[prompt_idx]

        # 랜덤 시드
        seed = random.randint(1, 99999999)

        print(f"\n시도 {attempt_count}/{max_attempts}")
        print(f"프롬프트: {current_prompt}")
        print(f"시드: {seed}")

        try:
            # 이미지 생성
            generated_image = alternative_face_generation(
                pipe=pipe,
                prompt=current_prompt,
                negative_prompt=negative_prompt,
                seed=seed,
                show_image=False  # 배치 처리시 화면 출력 끄기
            )

            if generated_image is None:
                continue

            # 유사도 평가
            scores = evaluator.evaluate_similarity(generated_image)
            total_score = scores['total_score']

            print(f"점수: {total_score:.3f} (얼굴: {scores['face_similarity']:.3f}, LPIPS: {scores['lpips_similarity']:.3f})")

            # 목표 점수 달성시 저장
            if total_score >= target_score:
                print(f"✅ 목표 달성! (점수: {total_score:.3f})")

                # 이미지 저장
                save_path = f"similar_face_{len(found_images)+1:02d}_score_{total_score:.3f}_seed_{seed}.png"
                generated_image.save(save_path)

                found_images.append({
                    'image': generated_image,
                    'scores': scores,
                    'prompt': current_prompt,
                    'seed': seed,
                    'path': save_path
                })

                # 이미지 표시
                plt.figure(figsize=(6, 6))
                plt.imshow(generated_image)
                plt.title(f"Found #{len(found_images)} (Score: {total_score:.3f})")
                plt.axis('off')
                plt.show()

        except Exception as e:
            print(f"오류 발생: {e}")
            continue

    print("\n" + "=" * 50)
    print(f"탐색 완료!")
    print(f"시도 횟수: {attempt_count}")
    print(f"발견된 이미지: {len(found_images)}개")

    if found_images:
        print("\n발견된 이미지 정보:")
        for i, result in enumerate(found_images):
            print(f"{i+1}. 점수: {result['scores']['total_score']:.3f}, 경로: {result['path']}")

    return found_images

print("이미지 생성 관리자 초기화 완료!")

In [ ]:
# 대량 재생성 함수
def regenerate_favorites(pipe, generation_ids):
    """즐겨찾기 이미지들 재생성"""
    results = []

    for gen_id in generation_ids:
        print(f"\n재생성 중: ID {gen_id}")
        image = gen_manager.regenerate_by_id(gen_id, pipe, save_new=True)
        if image:
            results.append({"id": gen_id, "image": image})

    return results

# 배치 생성 함수
def batch_generate_images(pipe, prompts, base_negative=None, seeds=None, save_folder=None):
    """여러 프롬프트로 배치 생성"""

    if save_folder is not None:
        os.makedirs(save_folder, exist_ok=True)

    if isinstance(prompts, str):
        prompts = [prompts]

    if seeds is None:
        seeds = [random.randint(1, 99999999) for _ in prompts]
    elif len(seeds) < len(prompts):
        seeds = seeds + [random.randint(1, 99999999) for _ in range(len(prompts) - len(seeds))]

    results = []

    for i, (prompt, seed) in enumerate(zip(prompts, seeds)):
        print(f"생성 {i+1}/{len(prompts)} (시드: {seed})")

        save_path = None
        if save_folder is not None:
            save_path = f"{save_folder}/image_{i+1:03d}_seed_{seed}.png"

        image = enhanced_face_generation(
            pipe=pipe,
            prompt=prompt,
            negative_prompt=base_negative,
            seed=seed,
            save_path=save_path,
            show_image=True
        )

        if image:
            results.append({
                "prompt": prompt,
                "seed": seed,
                "image": image,
                "path": save_path
            })

    return results

# 이미지 생성 시스템 사용 방법

## 1. 기본 이미지 생성 (히스토리 자동 저장)
```python
# 기본 생성
image = enhanced_face_generation(
    pipe=pipe,
    prompt="young Asian woman, professional portrait, natural lighting",
    negative_prompt="blurry, low quality, bad anatomy",
    seed=12345,
    save_path="test_portrait.png"
)
```

## 2. 본인 사진 기반 유사 이미지 자동 탐색
```python
# 참조 사진 경로 설정 (본인 사진)
reference_photo = "my_reference_photo.jpg"

# 자동 탐색 실행
similar_faces = find_similar_faces_auto(
    pipe=pipe,
    reference_image_path=reference_photo,
    target_score=0.65,  # 65% 이상 유사
    max_attempts=50,    # 최대 50번 시도
    target_count=3      # 3개 찾으면 종료
)
```

## 3. 히스토리 관리
```python
# 히스토리 보기
gen_manager.show_history(limit=5)

# 검색
results = gen_manager.search_generations(keyword="portrait")
seed_results = gen_manager.search_generations(seed=12345)

# 재생성
regenerated = gen_manager.regenerate_by_id(generation_id=3, pipe=pipe)
```

## 4. 배치 생성
```python
prompts = [
    "professional headshot, business attire",
    "casual portrait, outdoor lighting",
    "artistic portrait, dramatic lighting"
]

batch_results = batch_generate_images(
    pipe=pipe,
    prompts=prompts,
    base_negative="blurry, low quality, bad anatomy",
    save_folder="batch_portraits"
)
```

In [ ]:
# # 테스트: 기본 이미지 생성
# test_prompt = "young Asian woman, professional portrait, natural lighting, high quality"
# test_negative = "blurry, low quality, bad anatomy, deformed, cartoon, anime"

# test_image = enhanced_face_generation(
#     pipe=pipe,
#     prompt=test_prompt,
#     negative_prompt=test_negative,
#     seed=42,
#     save_path="test_generation.png"
# )

In [ ]:
# 간단한 실행 (통합 함수 재사용)
if helper.is_colab:
    reference_photo = "/content/drive/MyDrive/homepage/스프린트미션/스프린트미션_작업중/test.png"
else:
    reference_photo = "test.png"

# 파일 존재 확인
if not os.path.exists(reference_photo):
    print(f"❌ 참조 이미지를 찾을 수 없습니다: {reference_photo}")
    print("현재 디렉토리 파일 목록:")
    for file in os.listdir('.'):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"  - {file}")
else:
    print(f"✅ 참조 이미지 확인: {reference_photo}")

    # 통합 자동 탐색 실행 (기존 함수 재사용)
    try:
        similar_faces = find_similar_faces_auto_enhanced(
            pipe=pipe,
            reference_image_path=reference_photo,
            target_score=0.6,
            max_attempts=30,
            target_count=2,
            age_group="30대",
            gender="남성",
            ethnicity="백인",
            use_opencv_fallback=True  # 자동 백업 활성화
        )

        if similar_faces:
            print(f"\n🎉 {len(similar_faces)}개의 유사한 이미지를 성공적으로 찾았습니다!")

    except Exception as e:
        print(f"❌ 탐색 중 오류 발생: {e}")

        # 기본 생성 테스트 (기존 함수 재사용)
        print("기본 이미지 생성 테스트...")
        test_image = enhanced_face_generation(
            pipe=pipe,
            prompt="professional portrait, high quality, realistic",
            seed=42
        )